# Machine Learning Immediate Recurrence of Atrial Fibrillation (IRAF) Susceptibility

Immediate Recurrence of Atrial Fibrillation (IRAF) occurs after synchronized cardioversion with a prevalence of 5-26%. [[1]] Recurrence decreases cardioversion efficacy and puts patients through the unnecessary risk of unsuccessful cardioversion. Better knowledge of patient susceptibility factors will allow clinicians to pre-treat these patients to reduce IRAF incidence. This project aims to shed more light on IRAF susceptibility factors through analysis of a cardioversion database.

[1]: https://deepblue.lib.umich.edu/bitstream/handle/2027.42/72654/j.1540-8167.2003.02415.x.pdf?sequence=1&isAllowed=y

In [2]:
import pandas as pd

In [56]:
data_file = '../ml-iraf-data/cvdb_v1_sanp.xls'
cvdb = pd.read_excel(data_file, sheet_name = 1)

In [57]:
print(list(cvdb))

['seqno', 'ep_date', 'expiration_date SSDI 051904', 'FU date SSDI 051904', 'Status SSDI 051904', 'ep_date.1', 'FUDays SSDI 0504', 'echo_date1', 'test_type', 'timing', 'primary_rhythm', 'study_quality', 'left_atrium_cm', 'lvid_diastolic', 'lvid_systolic', 'eject_fraction', 'la_cavity_area_sys', 'la_cavity_area_dia', 'la_cavity_size_x', 'la_cavity_size_y', 'la_cavity_size_z', 'lv_cavity_dilation_size', 'lv_test_timing', 'lv_systolic_function', 'lv_diastolic_filling_baseline', 'rv_cavity_dilation_size', 'test_timing', 'rv_systolic_function', 'rv_diastolic_function', 'mitral_regurge', 'tricuspid_regurge', 'MR1Code', 'TR1Code', 'AR1Code', 'rvsp', 'aortic_regurge', 'pulmonic_regurge', 'echo_date2', 'test_type.1', 'timing.1', 'primary_rhythm.1', 'study_quality.1', 'left_atrium_cm.1', 'lvid_diastolic.1', 'lvid_systolic.1', 'eject_fraction.1', 'la_cavity_area_sys.1', 'la_cavity_area_dia.1', 'la_cavity_size_x.1', 'la_cavity_size_y.1', 'la_cavity_size_z.1', 'lv_cavity_dilation_size.1', 'lv_test_t

In [58]:
print(cvdb.shape)

(6849, 255)


In [61]:
print(pd.unique(cvdb['CV success']))

['SUCCESSFUL' 'UNSUCCESSFUL' 'TRANSIENTLY SUCCESSFUL']


In [65]:
sum(cvdb['CV success'] == 'TRANSIENTLY SUCCESSFUL')/cvdb.shape[0]

0.06176084099868594

In [70]:
print(cvdb.columns.get_loc('CV success'))

161


In [111]:
# One-hot encoding of CV success
from sklearn.preprocessing import LabelEncoder


### IRAF + Medications 

In [108]:
pd.unique(cvdb['Amiodarone'])

array([0, 1])

In [69]:
print(cvdb.columns.get_loc('Amiodarone'))
print(cvdb.columns.get_loc('Sotalol'))

193
209


In [109]:
cvdb_subset = list(range(193,209))
print(cvdb_subset)

[193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208]


In [110]:
cvdb_meds = cvdb.iloc[:,cvdb_subset]
list(cvdb_meds)

['Amiodarone',
 'Beta Blocker',
 'Ca+ channel blocker',
 'Digoxin',
 'Disopyramide',
 'Dofetelide',
 'Fentanyl',
 'Flecainide',
 'Lidocaine',
 'Mexiletine',
 'Moricizine',
 'None',
 'Phenytoin',
 'Procainamide',
 'Propafenone',
 'Quinidine']